# SAE Radio (Statistique descriptive / BD)

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

path=''
reponses=pd.read_csv(path+'reponses_radio_sauv.csv',sep=';')
data_audience = pd.read_csv(path+'audience_cumulee_radio.csv',sep=';', decimal=',', na_values=np.nan, skip_blank_lines=False) # decimal=',' nécessaire sinon convertie en str 

In [2]:
def showDF(df):
    print("\n", df.head(10), "\n")

In [3]:
showDF(reponses)


        Style musical    Radio
0                Pop  Radio 5
1  Musique classique  Radio 1
2      Hip-Hop & RnB  Radio 3
3      Hip-Hop & RnB  Radio 6
4               Rock  Radio 3
5            Variété  Radio 5
6                Pop  Radio 3
7            Electro  Radio 5
8      Hip-Hop & RnB  Radio 6
9               Jazz  Radio 2 



## Partie 1

In [4]:
def calcul_eff_obs():
    # .groupby().size() -> obtention de la valeur
    # .unstack()        -> déplace l'une des colone en index
    res = reponses.groupby(['Radio', 'Style musical']).size().unstack(level=0) # donnée ordonné
    
    showDF(res)
    return res # un dataframe

res_list_electro=[12,11,39,42,18,19]
res_list_radio3=[39, 46, 14, 19, 11, 31, 33, 28]
eff_obs=calcul_eff_obs()
print('test calcul_eff_obs : ',len(res_list_electro)-list(np.isclose(eff_obs.loc['Electro'],res_list_electro)).count(True)==0)
print('test calcul_eff_obs : ',len(res_list_radio3)-list(np.isclose(eff_obs['Radio 3'],res_list_radio3)).count(True)==0)



 Radio              Radio 1  Radio 2  Radio 3  Radio 4  Radio 5  Radio 6
Style musical                                                          
Electro                 12       11       39       42       18       19
Hip-Hop & RnB            4        6       46       18       31       58
Indie                    4        9       14        8       15        7
Jazz                    17       15       19        9       14        9
Musique classique       23       16       11        3        2        1
Pop                      5       27       31       17       37       67
Rock                    20       34       33       15       33       30
Variété                  6       54       28       16       20       27 

test calcul_eff_obs :  True
test calcul_eff_obs :  True


In [5]:
def calcul_eff_theo():
    tot_col = eff_obs.sum(axis=0)
    tot_row = eff_obs.sum(axis=1)
    tot_ech = eff_obs.sum().sum()
    
    res = pd.DataFrame(index=eff_obs.index, columns=eff_obs.columns)
    for style in res.index:
        for radio in res.columns:
            res.loc[style, radio] = (tot_col[radio]*tot_row[style])/tot_ech # calcul des valeurs une à une
    
    showDF(res)
    return res # un dataframe

res_list_indie=[5.187, 9.804, 12.597, 7.296, 9.69, 12.426]
res_list_radio6=[30.738, 35.534, 12.426, 18.094, 12.208, 40.112, 35.97, 32.918]
eff_theo=calcul_eff_theo()
print('test calcul_eff_theo : ',len(res_list_indie)-list(np.isclose(eff_theo.loc['Indie'].to_list(),res_list_indie)).count(True)==0)
print('test calcul_eff_theo : ',len(res_list_radio6)-list(np.isclose(eff_theo['Radio 6'].to_list(),res_list_radio6)).count(True)==0)



 Radio             Radio 1 Radio 2 Radio 3 Radio 4 Radio 5 Radio 6
Style musical                                                    
Electro            12.831  24.252  31.161  18.048   23.97  30.738
Hip-Hop & RnB      14.833  28.036  36.023  20.864   27.71  35.534
Indie               5.187   9.804  12.597   7.296    9.69  12.426
Jazz                7.553  14.276  18.343  10.624   14.11  18.094
Musique classique   5.096   9.632  12.376   7.168    9.52  12.208
Pop                16.744  31.648  40.664  23.552   31.28  40.112
Rock               15.015   28.38  36.465   21.12   28.05   35.97
Variété            13.741  25.972  33.371  19.328   25.67  32.918 

test calcul_eff_theo :  True
test calcul_eff_theo :  True


In [6]:
def calcul_contrib():
    # la formule est directement appliquer sur les 2 dataframe, pandas s'occupe du reste
    res = (eff_obs - eff_theo)**2/eff_theo
    
    showDF(res)
    return res # un dataframe

res_list_rock=[1.6550266400266396, 1.1129105003523614, 0.329253393665159, 1.7734090909090914, 0.8735294117647057, 0.9908507089241031]
res_list_radio1=[0.05381973345803127, 7.911675925301693, 0.2716346635820321, 11.815941877399705, 62.902907378335954, 8.237072145246058, 1.6550266400266396, 4.360896659631758]
contrib=calcul_contrib()
print('test calcul_contrib : ',len(res_list_rock)-list(np.isclose(contrib.loc['Rock'].to_list(),res_list_rock)).count(True)==0)
print('test calcul_contrib : ',len(res_list_radio1)-list(np.isclose(contrib['Radio 1'].to_list(),res_list_radio1)).count(True)==0)



 Radio                Radio 1    Radio 2   Radio 3    Radio 4   Radio 5  \
Style musical                                                            
Electro              0.05382   7.241279  1.972014  31.787362  1.486896   
Hip-Hop & RnB       7.911676  17.320063  2.763249   0.393141  0.390621   
Indie               0.271635   0.065934   0.15626    0.06793  2.909814   
Jazz               11.815942   0.036717  0.023532   0.248247  0.000858   
Musique classique  62.902907   4.210073  0.152988    2.42358  5.940168   
Pop                 8.237072   0.682631  2.296697    1.82272  1.045985   
Rock                1.655027   1.112911  0.329253   1.773409  0.873529   
Variété             4.360897  30.246757  0.864452   0.573033  1.252392   

Radio                Radio 6  
Style musical                 
Electro             4.482421  
Hip-Hop & RnB      14.203894  
Indie               2.369345  
Jazz                4.570622  
Musique classique  10.289913  
Pop                18.023647  
Rock     

In [7]:

def analyse_contrib(n):
    """
    renvoie une liste de tuples au format suivant :  
    (mod_style,mod_radio,signe,contrib) où signe sera représenté par le caractère '+'
    si l’effectif observé est supérieur à l’effectif théorique (tendance à une certaine attraction
    entre ces deux modalités) et par le caractère '-'si l’effectif observé est inférieur à l’effectif
    théorique (tendance à une certaine répulsion entre ces deux modalités)
    """
    res = []
    
    # contrib.unstack               -> passe les donnée (radio) en colone en index
    # .sort_values(ascending=False) -> trie les donnée pour avoir les valeurs les plus grande en haut
    # .head(n)                      -> prend le nombre n de donnée demandé
    temp = contrib.unstack().sort_values(ascending=False).head(n)
    showDF(temp)
    
    for i in temp.index :
        signe = ""
        if eff_obs.loc[i[1], i[0]] > eff_theo.loc[i[1], i[0]]: signe = "+"
        else : signe = "-"
        res.append((i[1], i[0], signe, temp.loc[i[0], i[1]]))
        
        # Todo demander à léa
        # style, radio = temp.index[i]
        # if eff_obs.loc[style, radio] > eff_theo.loc[style, radio]: signe = "+"
        # else : signe = "-"
        # res.append((style, radio, signe, temp.iloc[style, radio]))
        
    
    for i in res : print(i)
    print("")
    return res # une liste de tuples

ntest=5
ana_contrib=analyse_contrib(ntest)

res_list=[
    ('Musique classique', 'Radio 1', '+', 62.902907378335954), 
    ('Electro', 'Radio 4', '+', 31.78736170212767), 
    ('Variété', 'Radio 2', '+', 30.24675743107962), 
    ('Pop', 'Radio 6', '+', 18.023647387315513), 
    ('Hip-Hop & RnB', 'Radio 2', '-', 17.320063347125128)
]
test_contrib=(list(np.isclose([res_list[i][3] for i in range(ntest)],[ana_contrib[i][3] for i in range(ntest)])).count(True)==ntest)
test_sens_dep=([res_list[i][3]==ana_contrib[i][3] for i in range(ntest)].count(True)==ntest)
test_radios=([res_list[i][1]==ana_contrib[i][1] for i in range(ntest)].count(True)==ntest)
print('test analyse_contrib : ',test_contrib and test_sens_dep and test_radios)



 Radio    Style musical    
Radio 1  Musique classique    62.902907
Radio 4  Electro              31.787362
Radio 2  Variété              30.246757
Radio 6  Pop                  18.023647
Radio 2  Hip-Hop & RnB        17.320063
dtype: object 

('Musique classique', 'Radio 1', '+', 62.902907378335954)
('Electro', 'Radio 4', '+', 31.78736170212767)
('Variété', 'Radio 2', '+', 30.24675743107962)
('Pop', 'Radio 6', '+', 18.023647387315513)
('Hip-Hop & RnB', 'Radio 2', '-', 17.320063347125128)

test analyse_contrib :  True


## Partie 2

In [8]:
showDF(data_audience)


    Annee  Periode    AC
0   2016        1  80.6
1   2016        2  79.1
2   2016        3  79.7
3   2016        4  80.8
4   2017        1  80.2
5   2017        2  79.0
6   2017        3  79.5
7   2017        4  79.4
8   2018        1  79.5
9   2018        2  78.9 



In [10]:
def decomp_serie():
    # serie_mm = data_audience['AC'].rolling(window=4, center=True).mean() # skipna=True ne marche pas
    # serie_coef_sais = data_audience['AC'] / serie_mm # coef_sais = AC / mm
    # print(serie_mm, serie_coef_sais)
    
    rolling1 = data_audience['AC'].rolling(window=4, center=True).mean()
    rolling2 = data_audience['AC'].rolling(window=4, center=True).mean().shift(-1)
    mm = (rolling2 + rolling1)/2
   
    s1 = data_audience['AC'] / mm
    data_audience['S1'] = s1
    s2 = data_audience.groupby('Periode')['S1'].mean()
    meanS2 = s2.mean()
    s3 = s2 / meanS2

    serie_mm_df = pd.Series(mm)
    serie_coef_sais = pd.Series(s3)

    return serie_mm_df,serie_coef_sais
    # return serie_mm, serie_coef_sais # serie_mm, serie_coef_sais

res_moy_mob=[np.nan, np.nan, 80.0, 79.9375, 79.9, 79.7, 79.4375, 79.3375, 79.2125, 78.9875, 78.7, 78.35, 77.9125, 77.575, 77.4375, np.nan, np.nan, np.nan, np.nan, np.nan, 73.35, 73.26249999999999, 73.225, 73.1375, 72.7375, 71.925, 71.35, 71.125, 70.7375, 70.2375, np.nan, np.nan]
coef_sais={}
coef_sais['T2']=0.992730
coef_sais['T4']=1.006176
serie_mm,serie_coef_sais=decomp_serie()
print('test decomp_serie moy mob : ',list(np.isclose(np.nan_to_num(serie_mm.values),np.nan_to_num(np.array(res_moy_mob)))).count(True)==len(res_moy_mob) )
print('test decomp_serie coef sais : ',np.isclose(serie_coef_sais.loc[2],coef_sais['T2']) and np.isclose(serie_coef_sais.loc[4],coef_sais['T4']))


test decomp_serie moy mob :  True
test decomp_serie coef sais :  True


In [ ]:
def prevision(annee_prev,trim_prev,serie_coef_sais):
    return # une valeur

prev=prevision(2024,4,serie_coef_sais)
res=68.7763620276032
print('test prévision : ',np.isclose(prev,res))